In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_classic.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate
from langchain_classic.memory import ConversationSummaryBufferMemory
from langchain_core.runnables import RunnablePassthrough

llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, max_tokens=300)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

examples = [
    {"movie_title": "Top Gun",
    "answer": "🛩️👨‍✈️🔥"},
    {"movie_title": "The Godfather",
    "answer": "👨‍👨‍👦🔫🍝"}
]

example_prompt = ChatPromptTemplate([
    ("human", "{movie_title}"),
    ("ai","{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You will get the title of movie. You shold reply with three emojis that represent the movie."),
    example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human","{movie_title}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(movie_title):
    result = chain.invoke({
    "movie_title": movie_title
    })
    memory.save_context({"input": movie_title}, {"ouput":result.content})
    print(result)


In [44]:
invoke_chain("Avatar")


content='🌿🌌💙' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 87, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqhzBtFD0ENjVB9376ApbopeV4m0B', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5639-93a0-7ec2-be47-0c06a553d38b-0' usage_metadata={'input_tokens': 87, 'output_tokens': 8, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [45]:
invoke_chain("Titanic")


content='🚢💔❄️' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 105, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqhzWMYh2FYhqVx982gXZ04IQQDQJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5639-e6b0-7d71-877a-2c13631f301a-0' usage_metadata={'input_tokens': 105, 'output_tokens': 8, 'total_tokens': 113, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [46]:
invoke_chain("zootopia")


content='🐰🦊🐾' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 124, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqhzuT6EXkC0Kq3aYEqb1hkh1zwUq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b563a-42e0-75a2-a169-017fb50c7c78-0' usage_metadata={'input_tokens': 124, 'output_tokens': 9, 'total_tokens': 133, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [47]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Avatar', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🌿🌌💙', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Titanic', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🚢💔❄️', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='zootopia', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🐰🦊🐾', additional_kwargs={}, response_metadata={})]}

In [48]:
invoke_chain("Parasite")

content='🏠👨\u200d👩\u200d👧\u200d👦💰' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 144, 'total_tokens': 167, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cqi8PbLQQCy6dwJlxPG5M5LSxdvl9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5642-4fe8-7fe0-a44f-f917a6dab72b-0' usage_metadata={'input_tokens': 144, 'output_tokens': 23, 'total_tokens': 167, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [49]:
invoke_chain("Notebook")

content='📔💑☔️' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 177, 'total_tokens': 185, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cqi8uducZarNCXoG1a0Y4tLrJmqiC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5642-c5ea-7f31-bc2c-9309b813c92d-0' usage_metadata={'input_tokens': 177, 'output_tokens': 8, 'total_tokens': 185, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [50]:
invoke_chain("star is born")


content='🌟🎤💔' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 196, 'total_tokens': 204, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cqi9DKVAgl2nS86ZCmQ54Hs530kBp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5643-1282-7793-a88a-3d36365f87b1-0' usage_metadata={'input_tokens': 196, 'output_tokens': 8, 'total_tokens': 204, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [51]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human mentions "Avatar" to the AI, and the AI responds with emojis representing nature, the cosmos, and a heart.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Titanic', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🚢💔❄️', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='zootopia', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🐰🦊🐾', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Parasite', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🏠👨\u200d👩\u200d👧\u200d👦💰', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Notebook', additional_kwargs={}, response_metadata={}),
  AIMessage(content='📔💑☔️', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='star is born', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🌟🎤💔', additional_kwargs={}, response_metadata={})]}

In [52]:
invoke_chain("Lionking")


content='🦁👑🌅' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 228, 'total_tokens': 237, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqiA8VTI2w85pq5kpbRnwSiybUojv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5643-f050-7a30-9608-55ff06b7a8de-0' usage_metadata={'input_tokens': 228, 'output_tokens': 9, 'total_tokens': 237, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [53]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human mentions "Avatar" to the AI, and the AI responds with emojis representing nature, the cosmos, and a heart. When the human brings up "Titanic," the AI responds with emojis of a ship, a broken heart, and snowflakes.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='zootopia', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🐰🦊🐾', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Parasite', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🏠👨\u200d👩\u200d👧\u200d👦💰', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Notebook', additional_kwargs={}, response_metadata={}),
  AIMessage(content='📔💑☔️', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='star is born', additional_kwargs={}, response_metadata={}),
  AIMessage(content='🌟🎤💔', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Lionking', additional_kwargs={}, re